In [ ]:
prompt = "What is the best temperature for matcha"

In [ ]:
documents = ["Use 70–80°C water and whisk briskly in an M-shape for 15–20 seconds to create fine foam and minimize bitterness",
             "Sift 1–2 g matcha, add ~60–70 ml at ~75°C, then whisk rapidly until microfoam forms and no clumps remain",
             "Use boiling water to fully extract flavor, then stir slowly to avoid bubbles.",
             "Use any whisk; forks work as well as chasen if whisked gently.",
             "GPU tensor cores speed up attention by enabling mixed precision matmul"
             ]

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
prompt_embed = model.encode(prompt)

In [ ]:
doc_embed = model.encode(documents)
len(doc_embed[0])

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
sim = cosine_similarity(np.array([prompt_embed]), doc_embed)
top_index = np.argmax(sim)
top_doc = documents[top_index]
print(f"Prompt : {prompt}")
print(f"Top Document : {top_doc}")

In [ ]:
sorted = np.argsort(sim[0][::-1])

ranking = [(documents[i], sim[0][i]) for i in sorted]
ranking

:NOW LET'S LOOK AT USING CROSS_ENCODER TO IMPROVE RAG VIA RE-RANKING
https://www.sbert.net/examples/applications/cross-encoder/README.html


In [ ]:
from sentence_transformers import CrossEncoder
# cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')

In [ ]:
pairs = []
for doc in documents:
  pairs.append([prompt, doc])

In [ ]:
scores = cross_encoder.predict(pairs)

print(f"Prompt {prompt}\n")
for doc, score in zip(documents, scores):
  print(f"Document: {doc} \nScore: {score}\n")

In [ ]:
top_index_cross = np.argmax(scores)
top_doc_cross = documents[top_index_cross]
print(f"Prompt : {prompt}")
print(f"Top Document : {top_doc_cross}")

In [ ]:
!pip install cohere

In [ ]:
import cohere
co = cohere.Client("API_KEY")

response = co.rerank(
    model="rerank-english-v3.0",
    query=prompt,
    documents=documents,
    return_documents=True
)

print(response)